# Интеллектуальный анализ данных – весна 2025
# Домашнее задание 6: классификация текстов

Правила:



*   Домашнее задание оценивается в 10 баллов.
*   Можно использовать без доказательства любые результаты, встречавшиеся на лекциях или семинарах по курсу, если получение этих результатов не является вопросом задания.
*  Можно использовать любые свободные источники с *обязательным* указанием ссылки на них.
*  Плагиат не допускается. При обнаружении случаев списывания, 0 за работу выставляется всем участникам нарушения, даже если можно установить, кто у кого списал.
*  Старайтесь сделать код как можно более оптимальным. В частности, будет штрафоваться использование циклов в тех случаях, когда операцию можно совершить при помощи инструментов библиотек, о которых рассказывалось в курсе.
* Если в задании есть вопрос на рассуждение, то за отсутствие ответа на него балл за задание будет снижен вполовину.

В этом домашнем задании вам предстоит построить классификатор текстов.

Будем предсказывать эмоциональную окраску твиттов о коронавирусе.



In [43]:
import numpy as np
import pandas as pd
from typing import  List
import matplotlib.pyplot as plt
import seaborn as sns
from string import punctuation

In [44]:
df = pd.read_csv('tweets_coronavirus.csv', encoding='latin-1')
df.sample(4)

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
24253,33457,78409,"Arkansas, USA",05-04-2020,"I better not see any gyms, dieticians, persona...",Positive
16409,23779,68731,Canada,24-03-2020,"Coronavirus will drive paper, market pulp, for...",Negative
7007,12320,57272,"Lagos, Nigeria",19-03-2020,4 of 5 ...how does the #Coronavirus affect tho...,Positive
30729,41598,86550,4 8 15 16 23 42,11-04-2020,So when we see stories about food insecurity i...,Extremely Negative


Для каждого твитта указано:


*   UserName - имя пользователя, заменено на целое число для анонимности
*   ScreenName - отображающееся имя пользователя, заменено на целое число для анонимности
*   Location - местоположение
*   TweetAt - дата создания твитта
*   OriginalTweet - текст твитта
*   Sentiment - эмоциональная окраска твитта (целевая переменная)



## Задание 1 Подготовка (0.5 балла)

Целевая переменная находится в колонке `Sentiment`.  Преобразуйте ее таким образом, чтобы она стала бинарной: 1 - если у твитта положительная или очень положительная эмоциональная окраска и 0 - если отрицательная или очень отрицательная.

In [45]:
df["Sentiment"] = np.where(df["Sentiment"].str.contains("Positive"), 1, 0)
df.head()

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,3800,48752,UK,16-03-2020,advice Talk to your neighbours family to excha...,1
1,3801,48753,Vagabonds,16-03-2020,Coronavirus Australia: Woolworths to give elde...,1
2,3802,48754,NaN,16-03-2020,My food stock is not the only one which is emp...,1
3,3803,48755,NaN,16-03-2020,"Me, ready to go at supermarket during the #COV...",0
4,3804,48756,"ÃÂT: 36.319708,-82.363649",16-03-2020,As news of the regionÃÂs first confirmed COV...,1


Сбалансированы ли классы?

In [46]:
df["Sentiment"].value_counts()

Sentiment
1    18046
0    15398
Name: count, dtype: int64

In [47]:
15398/18046 * 100

85.32638811925081

**Ответ:** При разнице менее 15% можно мы можем принять классы из колонки Sentiment как сбалансированные

Выведете на экран информацию о пропусках в данных. Если пропуски присутствуют заполните их строкой 'Unknown'.

In [48]:
df.isna().sum()

UserName            0
ScreenName          0
Location         7049
TweetAt             0
OriginalTweet       0
Sentiment           0
dtype: int64

In [49]:
df["Location"] = df["Location"].fillna("Unknown")
df.isna().sum()

UserName         0
ScreenName       0
Location         0
TweetAt          0
OriginalTweet    0
Sentiment        0
dtype: int64

In [50]:
df.sample(10)

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
585,4495,49447,"Strensall, North Yorkshire, UK",17-03-2020,407 new confirmed cases in the UK Please remem...,0
15434,22577,67529,"Amelia Island, FL",23-03-2020,"In response to the #Covid_19 crisis, we've low...",0
8095,13635,58587,London,20-03-2020,The Space Daddy is as disappointed as I am in ...,0
10214,16197,61149,Unknown,21-03-2020,Missouri woman gives birth in the #toiletpaper...,1
5833,10886,55838,"Toronto, Canada",19-03-2020,The outbreak of COVID 19 along with the dine i...,0
21940,30577,75529,"Waterloo, Ontario, Canada",02-04-2020,"""Meet Michael Gill, the man in charge of coron...",1
25744,35335,80287,Unknown,07-04-2020,Supermarket shoppers in S pore ditch baskets f...,0
31405,42440,87392,United Kingdom,11-04-2020,I love being able to learn how to cook more me...,1
231,4078,49030,On Your Timeline,16-03-2020,"@Amazon says it needs to hire 100,000 people a...",0
23541,32556,77508,Gold Coast,05-04-2020,@GerardDaffy I agree Gerard. I followed a coup...,0


Разделите данные на обучающие и тестовые в соотношении 7 : 3 и укажите `random_state=0`

In [51]:
from sklearn.model_selection import train_test_split

y = df["Sentiment"]
X = df.drop(["Sentiment"], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

## Задание 2 Токенизация (3 балла)

Постройте словарь на основе обучающей выборки и посчитайте количество встреч каждого токена с использованием самой простой токенизации - деления текстов по пробельным символам и приведения токенов в нижний регистр.

In [116]:
tweets = X_train["OriginalTweet"]
dictionary = dict()

for tweet in tweets:
    for word in tweet.lower().split():
        dictionary[word] = dictionary.get(word, 0) + 1

Какой размер словаря получился?

In [63]:
len(dictionary)

79755

Выведите 10 самых популярных токенов с количеством встреч каждого из них. Объясните, почему именно эти токены в топе.

In [69]:
dictionary_sorted = pd.Series(dictionary)
dictionary_sorted = dictionary_sorted.sort_values(ascending=False)
dictionary_sorted.head(10)

the             26815
to              23373
and             14684
of              13012
a               11737
in              11198
for              8566
#coronavirus     8223
is               7383
are              7050
dtype: int64

**Ответ:** Это предлоги, союзы, частицы и прочие слова, которые часто встречаются в предложениях и имеют только одну форму, что увеличивает частоту их появлений. Кроме того, эти слова являются стоп-словами, так как не несут смысловой нагрузки, а только связывают другие слова в предлжении

Удалите стоп-слова из словаря и выведите новый топ-10 токенов (и количество встреч) по популярности.  Что можно сказать  о нем?

In [76]:
import nltk
# nltk.download('stopwords')
from nltk.corpus import stopwords

stop_words = stopwords.words("english")
new_dictionary = {key: value for key, value in dictionary.items() if key not in stop_words}

no_stop_words = pd.Series(new_dictionary)
no_stop_words = no_stop_words.sort_values(ascending=False)
no_stop_words.head(10)

#coronavirus    8223
prices          3891
food            3820
grocery         3469
supermarket     3288
people          3175
covid-19        3173
store           3155
#covid19        2471
&amp;           2314
dtype: int64

**Ответ:**  Теперь из топа пропали служебные части речи, остались только слова имеющие смысл, что гораздо ценнее для нас при работе с текстом. Судя по словам, датасет был собран в 2020-2021 году, главная тема для обсуждения с огромным отрывом - ковид, кроме него люди обсуждаю обычный быт - цены, еду и магазины

Также выведите 20 самых непопулярных слов (если самых непопулярных слов больше, выведите любые 20 из них) Почему эти токены непопулярны, требуется ли как-то дополнительно работать с ними?

In [77]:
no_stop_words.tail(20)

466                        1
pmt                        1
acres                      1
cucumber                   1
https://t.co/x69jqbisox    1
had!                       1
massive.                   1
https://t.co/6ytxpz5ug9    1
https://t.co/l8jnzxjgwo    1
readiness,                 1
kick-in                    1
(covid-19,                 1
impacts)                   1
https://t.co/wa7kcdwqea    1
@linkedin                  1
clare                      1
connors                    1
levins                     1
587-4272                   1
https://t.co/7j2y3rsld9    1
dtype: int64

**Ответ:** Самые непопулярные токены - это ссылки и необычные формы слов. На этом примере хорошо видно проблему такого токенизатора, он не учитывает формы слов и соседние символы, например, в этом списке есть "(covid-19,". Тема ковида - одна из популярнейших, но комбинация соседних символов вывела этот токен вниз. Для улучшения работы модели это нужно исправить



Теперь воспользуемся токенайзером получше - TweetTokenizer из библиотеки nltk. Примените его и посмотрите на топ-10 популярных слов. Чем он отличается от топа, который получался раньше? Почему?

In [122]:
from nltk.tokenize import TweetTokenizer

tweet_tokenizer = TweetTokenizer()
dictionary = dict()

for tweet in tweets:
    tokens = tweet_tokenizer.tokenize(tweet)
    for token in tokens:
        dictionary[token.lower()] = dictionary.get(token.lower(), 0) + 1

dictionary_sorted = pd.Series(dictionary)
dictionary_sorted = dictionary_sorted.sort_values(ascending=False)
dictionary_sorted.head(10)

the             26993
.               24118
to              23478
,               17571
and             14825
of              13044
a               11891
in              11348
?                9524
#coronavirus     8808
dtype: int64

**Ответ:** В отличие от простого токенизатора, теперь в топ, кроме вспомогательных частей речи попали символы, которые ранее были частью слов. Теперь они стали полноценными токенами, и очень популярными

Удалите из словаря стоп-слова и пунктуацию, посмотрите на новый топ-10 слов с количеством встреч, есть ли теперь в нем что-то не похожее на слова?

In [123]:
from string import punctuation

new_dictionary = {key: value for key, value in dictionary.items() if (key not in stop_words) and (key not in list(punctuation))}
no_stop_words = pd.Series(new_dictionary)
no_stop_words = no_stop_words.sort_values(ascending=False)
no_stop_words.head(10)

#coronavirus    8808
â               7415
               7311
19              7167
covid           6253
prices          4601
               4372
food            4367
store           3877
supermarket     3805
dtype: int64

**Ответ:** Теперь топ-10 похож на тот,который получали ранее с обычным токенизатором: ковид и бытовые тема, но также несколько странных и неотображаемых символов

Скорее всего в некоторых топах были неотображаемые символы или отдельные буквы не латинского алфавита. Уберем их: удалите из словаря токены из одного символа, позиция которого в таблице Unicode 128 и более (`ord(x) >= 128`)

Выведите топ-10 самых популярных и топ-20 непопулярных слов. Чем полученные топы отличаются от итоговых топов, полученных при использовании токенизации по пробелам? Что теперь лучше, а что хуже?

In [126]:
def filt(x):
    if len(x) <= 1:
        if ord(x) >= 128:
            return False
    return True

new_new_dictionary = {key: value for key, value in new_dictionary.items() if filt(key)}
final_dict = pd.Series(new_new_dictionary)
final_dict = final_dict.sort_values(ascending=False)
final_dict.head(10)

#coronavirus    8808
19              7167
covid           6253
prices          4601
food            4367
store           3877
supermarket     3805
grocery         3523
people          3463
#covid19        2589
dtype: int64

In [127]:
final_dict.tail(20)

@awgcorporate              1
https://t.co/l8jnzxjgwo    1
587-4272                   1
readiness                  1
kick-in                    1
https://t.co/wa7kcdwqea    1
@linkedin                  1
clare                      1
connors                    1
levins                     1
https://t.co/6ytxpz5ug9    1
https://t.co/8xikga3rel    1
quantitatively             1
logarithmic                1
non-log                    1
improveã                   1
https://t.co/aacxqg5sej    1
rebooked                   1
can't-miss                 1
https://t.co/7j2y3rsld9    1
dtype: int64

**Ответ:** топ остался практически таким же, только "19" стал отдельным символом. А вот низ рейтинга стал гораздо полезнее для наших целей, теперь там нет необычных комбинаций из слов и знаков пунктуации, теперь тут ссылки и редко используемые слова

Выведите топ-10 популярных хештегов (токены, первые символы которых - #) с количеством встреч. Что можно сказать о них?

In [132]:
hashtag = {key: value for key, value in new_new_dictionary.items() if key[0] == "#"}
hashtags = pd.Series(hashtag)
hashtags = hashtags.sort_values(ascending=False)
hashtags.head(10)

#coronavirus            8808
#covid19                2589
#covid_19               1734
#covid2019               946
#toiletpaper             744
#covid                   641
#socialdistancing        465
#coronacrisis            448
#pandemic                257
#coronaviruspandemic     249
dtype: int64

**Ответ:** Выводы остаются примерно теми же, ковид и бытовые темы, а также призывы к действию (socialdistancing)

То же самое проделайте для ссылок на сайт https://t.co Сравнима ли популярность ссылок с популярностью хештегов? Будет ли информация о ссылке на конкретную страницу полезна?

In [134]:
link = {key: value for key, value in new_new_dictionary.items() if key.startswith("https://t.co")}
links = pd.Series(link)
links = links.sort_values(ascending=False)
links.head(10)

https://t.co/oxa7swtond    5
https://t.co/gp3eusapl8    4
https://t.co/kuwipf1kqw    3
https://t.co/bylqxrjmnt    3
https://t.co/wuieefsnoj    3
https://t.co/zjnrx6dkkn    3
https://t.co/wrlhyzizaa    3
https://t.co/g63rp042ho    3
https://t.co/e2znxajpre    3
https://t.co/catkegayoy    3
dtype: int64

**Ответ:** Ссылки используются ГОРАЗДО реже хештегов, одна ссылка участвует в 5 постах, что не позволяет использовать их для поиска зависимостей

Используем опыт предыдущих экспериментов и напишем собственный токенайзер, улучшив TweetTokenizer. Функция tokenize должна:



*   Привести текст в нижний регистр
*   Применить TweetTokenizer для  выделения токенов
*   Удалить стоп-слова, пунктуацию, токены из одного символа с позицией в таблице Unicode 128 и более,  ссылки на t.co



In [139]:
def filt(x):
    if len(x) <= 1:
        if ord(x) >= 128:
            return False
    if x.startswith("https://t.co"):
        return False
    if x in stop_words:
        return False
    if x in list(punctuation):
        return False
    return True

def custom_tokenizer(text):

  tokens = tweet_tokenizer.tokenize(text.lower())

  return [token for token in tokens if filt(token)]


In [140]:
custom_tokenizer('This is sample text!!!! @Sample_text I, \x92\x92 https://t.co/sample  #sampletext')

['sample', 'text', '@sample_text', '#sampletext']

## Задание 3 Векторизация текстов (2 балла)

Обучите CountVectorizer с использованием custom_tokenizer в качестве токенайзера. Как размер полученного словаря соотносится с размером изначального словаря из начала задания 2?

In [141]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(tokenizer=custom_tokenizer)
cv.fit_transform(X_train["OriginalTweet"])

print(len(cv.vocabulary_))

C:\Users\prusakovR\anaconda3\Lib\site-packages\sklearn\feature_extraction\text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


45290


**Ответ:** # было 79755, стал почти в 2 раза меньше

Посмотрим на какой-нибудь конкретный твитт:

In [147]:
ind = 9023
df.iloc[ind]['OriginalTweet'], df.iloc[ind]['Sentiment']

("Shop keepers taking advantage of #Coronavirus boosting prices disproportionately so the most Marginalised suffer who can't afford it #SHAMEONYOU #Wewillremember",
 0)

Автор твитта не доволен ситуацией с едой во Франции и текст имеет резко негативную окраску.

Примените обученный CountVectorizer для векторизации данного текста, и попытайтесь определить самый важный токен и самый неважный токен (токен, компонента которого в векторе максимальна/минимальна, без учета 0). Хорошо ли они определились, почему?

In [148]:
france_eat = cv.transform([df.iloc[ind]['OriginalTweet']]).toarray()
tokens = pd.Series(france_eat.reshape(-1), index=cv.get_feature_names_out())
tokens = tokens.sort_values(ascending=False)
tokens.head(20)

#wewillremember       1
can't                 1
shop                  1
afford                1
disproportionately    1
#coronavirus          1
boosting              1
prices                1
#shameonyou           1
suffer                1
taking                1
marginalised          1
keepers               1
advantage             1
interest              0
interactions          0
intereste             0
interesed             0
interested            0
interesting           0
dtype: int64

**Ответ:** Векторизация дала одинаковый вес всем токенам в тексте, что не помогает нам в поиске самого важного и неважного токена

Теперь примените TfidfVectorizer и  определите самый важный/неважный токены. Хорошо ли определились, почему?

In [149]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(tokenizer=custom_tokenizer)
tfidf.fit(X_train["OriginalTweet"])

france_eat = tfidf.transform([df.iloc[ind]['OriginalTweet']]).toarray()
tokens = pd.Series(france_eat.reshape(-1), index=cv.get_feature_names_out())
tokens = tokens.sort_values(ascending=False)
tokens.head(20)

C:\Users\prusakovR\anaconda3\Lib\site-packages\sklearn\feature_extraction\text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


#wewillremember       0.373460
marginalised          0.373460
#shameonyou           0.348492
keepers               0.333887
disproportionately    0.328334
boosting              0.306035
suffer                0.266462
afford                0.230787
advantage             0.211172
can't                 0.196633
taking                0.187111
shop                  0.172201
prices                0.096845
#coronavirus          0.071426
interactive           0.000000
intercept             0.000000
intercession          0.000000
interconnected        0.000000
interests             0.000000
interesed             0.000000
dtype: float64

**Ответ:** Такая векторизация показывает себя гораздо лучше, самый бесполезный токен - "#coronavirus", так как он встричается очень часто и не дает никакой подсказки об эмоциональной окраске твита, а вот самые популярные токены показывают сильный негативный оттенок. Вывод - TFIDF токенизация действительно хорошо сказывается на эффективности нашей модели

Найдите какой-нибудь положительно окрашенный твитт, где TfidfVectorizer хорошо (полезно для определения окраски) выделяет важный токен, поясните пример.

*Подсказка:* явно положительные твитты можно искать при помощи положительных слов (good, great, amazing и т. д.)

In [171]:
df[df['OriginalTweet'].apply(lambda x: 'fantastic' in x) & (df['Sentiment'] == 1)]

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
242,4091,49043,UK,16-03-2020,Loving how quiet the Tube is atm. COVID-19 is ...,1
1760,5946,50898,"West Midlands, UK",17-03-2020,@mcdonalds your free drink for social and emer...,1
2631,6980,51932,Unknown,18-03-2020,@TheSun The panic buying will continue for wee...,1
3429,7962,52914,"Spalding, England",18-03-2020,Healthcare workers are fantastic! I would also...,1
3964,8619,53571,San Francisco,18-03-2020,@AmyGorin shares a fantastic list of foods rec...,1
4707,9517,54469,"London, England",19-03-2020,Thanks to great teamwork from colleagues at @W...,1
5645,10654,55606,"Coventry, ENG ???????",19-03-2020,@drphilhammond to clarify: our pub is fantasti...,1
9322,15113,60065,"Toronto, Ontario",20-03-2020,For all the grocery store workers during this ...,1
15864,23086,68038,Cumbria & South West Scotland,24-03-2020,NEW DEALS AVAILABLE EVERYDAY When browsing our...,1
16627,24042,68994,"Saskatoon, Canada",24-03-2020,Absolutely fantastic TEDx Talk.\r\r\nWhy are p...,1


In [174]:
tweets = custom_tokenizer(df.iloc[2631]['OriginalTweet'])
result = tfidf.transform([df.iloc[2631]['OriginalTweet']]).toarray()

result = pd.Series(result.reshape(-1), index=tfidf.get_feature_names_out())
result = result.reindex(tweets)
result = result.sort_values(ascending=False)
result.head(20)

turnover        0.467661
@thesun         0.454767
fantastic       0.440007
#panicbuying    0.297489
continue        0.280454
weeks           0.256627
#covid2019      0.215935
buying          0.210145
panic           0.189423
supermarket     0.145574
dtype: float64

**Ответ:** Взял 2631-ый твит. Я выбрал слово fantastic, как слово с явно позитивной окраской, проиндксировав выбранный твит, мое слово и правда стало одним из 3-ех самых важных токенов, однако токены turnover и @thesun важнее, чем fantastic. Я полагаю, что это связано с распространенностью данного токена, поэтому более конкретные слова оказывают больше влияния на вывод векторизатора

## Задание 4 Обучение первых моделей (1 балл)

Примените оба векторайзера для получения матриц с признаками текстов.  Выделите целевую переменную.

In [187]:
cv_train = cv.transform(X_train['OriginalTweet'])
cv_test = cv.transform(X_test['OriginalTweet'])

tfidf_train = tfidf.transform(X_train['OriginalTweet'])
tfidf_test = tfidf_vec.transform(X_test['OriginalTweet'])

NameError: name 'tfidf_vec' is not defined

Обучите логистическую регрессию на векторах из обоих векторайзеров. Посчитайте долю правильных ответов на обучающих и тестовых данных. Какой векторайзер показал лучший результат? Что можно сказать о моделях?

Используйте `sparse` матрицы (после векторизации), не превращайте их в `numpy.ndarray` или `pd.DataFrame` - может не хватить памяти.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

cv_regression = LogisticRegression()
tfidf_regression = LogisticRegression()

cv_regression.fit(cv_train, y_train)
tfidf_regression.fit(tfidf_train, y_train)

cv_train_pred = cv_regression.predict(cv_train)
tfidf_train_pred = tfidf_regression.predict(tfidf_train)

cv_test_pred = cv_regression.predict(cv_test)
tfidf_test_pred = tfidf_regression.predict(tfidf_test)

print("cv_train_pred", accuracy_score(cv_train_pred, y_train))
print("cv_test_pred", accuracy_score(cv_test_pred, y_test))
print()
print("tfidf_train_pred", accuracy_score(tfidf_train_pred, y_train))
print("tfidf_test_pred", accuracy_score(tfidf_test_pred, y_test))

**Ответ:** # На нашем датасете лучше себя показывает более простая модель регрессии, полагаю, что это связано с простотой задачи, tfidf просто не успевает раскрыться 

## Задание 5 Стемминг (0.5 балла)

Для уменьшения словаря можно использовать стемминг.

Модифицируйте написанный токенайзер, добавив в него стемминг с использованием SnowballStemmer. Обучите Count- и Tfidf- векторайзеры. Как изменился размер словаря?

In [183]:
from nltk.tokenize import TweetTokenizer
from nltk.stem.snowball import SnowballStemmer
from string import punctuation

tokenizer = TweetTokenizer()
stemmer = SnowballStemmer("english")

def custom_stem_tokenizer(text):
    tokens = []
    stem = [stemmer.stem(t) for t in tokenizer.tokenize(text)]
    
    for x in stem:
        if len(x) <= 1:
            if ord(x) >= 128:
                continue
        if x.startswith("https://t.co"):
            continue
        if x in stop_words:
            continue
        if x in list(punctuation):
            continue

        tokens.append(x)

    return tokens

In [184]:
custom_stem_tokenizer('This is sample text!!!! @Sample_text I, \x92\x92 https://t.co/sample  #sampletext adding more words to check stemming')

['sampl', 'text', '@sample_text', '#sampletext', 'ad', 'word', 'check', 'stem']

In [186]:
cv = CountVectorizer(tokenizer=custom_stem_tokenizer)
cv.fit(X_train["OriginalTweet"])

print(len(cv.vocabulary_))

36608


**Ответ** # Словарь стал меньше, так как уменьшилось кол-во форм одних и тех же слов, так как при стемминге удаляется аффикс

Обучите логистическую регрессию с использованием обоих векторайзеров. Изменилось ли качество? Есть ли смысл применять стемминг?

In [189]:
count_vc_stemed = CountVectorizer(tokenizer=custom_stem_tokenizer)
tfidf_vc_stemed = TfidfVectorizer(tokenizer=custom_stem_tokenizer)

count_vc_stemed.fit(X_train["OriginalTweet"])
tfidf_vc_stemed.fit(X_train["OriginalTweet"])

cv_test = count_vc_stemed.transform(X_test["OriginalTweet"])
tfidf_test = tfidf_vc_stemed.transform(X_test["OriginalTweet"])

cv_train = count_vc_stemed.transform(X_train["OriginalTweet"])
tfidf_train = tfidf_vc_stemed.transform(X_train["OriginalTweet"])

cv_regression = LogisticRegression()
tfidf_regression = LogisticRegression()

cv_regression.fit(cv_train, y_train)
tfidf_regression.fit(tfidf_train, y_train)

cv_train_pred = cv_regression.predict(cv_train)
tfidf_train_pred = tfidf_regression.predict(tfidf_train)

cv_test_pred = cv_regression.predict(cv_test)
tfidf_test_pred = tfidf_regression.predict(tfidf_test)

print("cv_train_pred", accuracy_score(cv_train_pred, y_train))
print("cv_test_pred", accuracy_score(cv_test_pred, y_test))
print()
print("tfidf_train_pred", accuracy_score(tfidf_train_pred, y_train))
print("tfidf_test_pred", accuracy_score(tfidf_test_pred, y_test))

C:\Users\prusakovR\anaconda3\Lib\site-packages\sklearn\feature_extraction\text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


cv_train_pred 0.9717214865442119
cv_test_pred 0.8681482957942994

tfidf_train_pred 0.9155489107219137
tfidf_test_pred 0.8546940402631055


**Ответ:** В данном примере использовать стемминг нет смысла, так как прироста к качеству работы модели нет

## Задание  6 Работа с частотами (1.5 балла)

Еще один способ уменьшить количество признаков - это использовать параметры min_df и max_df при построении векторайзера  эти параметры помогают ограничить требуемую частоту встречаемости токена в документах.

По умолчанию берутся все токены, которые встретились хотя бы один раз.



Подберите max_df такой, что размер словаря будет 36651 (на 1 меньше, чем было). Почему параметр получился такой большой/маленький?

In [ ]:
cv_df = CountVectorizer(tokenizer=custom_stem_tokenizer,
                        max_df=# -- YOUR CODE HERE --
                        ).fit(
                            # -- YOUR CODE HERE --
                            )
print(len(cv_df.vocabulary_))

36651


In [ ]:
# -- YOUR CODE HERE --

**Ответ:** # -- YOUR ANSWER HERE --

Подберите min_df (используйте дефолтное значение max_df) в CountVectorizer таким образом, чтобы размер словаря был 3700 токенов (при использовании токенайзера со стеммингом), а качество осталось таким же, как и было. Что можно сказать о результатах?

In [ ]:
cv_df = CountVectorizer(tokenizer=custom_stem_tokenizer,
                        min_df=# -- YOUR CODE HERE --
                        ).fit(
                            # -- YOUR CODE HERE --
                            )
print(len(cv_df.vocabulary_))

3700


In [ ]:
# -- YOUR CODE HERE --

**Ответ:** # -- YOUR ANSWER HERE --

В предыдущих заданиях признаки не скалировались. Отскалируйте данные (при словаре размера 3.7 тысяч, векторизованные CountVectorizer), обучите логистическую регрессию, посмотрите качество и выведите `barplot`, содержащий по 10 токенов, с наибольшим по модулю положительными/отрицательными весами. Что можно сказать об этих токенах?

In [ ]:
from sklearn.preprocessing import StandardScaler
# -- YOUR CODE HERE --

**Ответ:** # -- YOUR ANSWER HERE --

## Задание 7 Другие признаки (1.5 балла)

Мы были сконцентрированы на работе с текстами твиттов и не использовали другие признаки - имена пользователя, дату и местоположение

Изучите признаки UserName и ScreenName. полезны ли они? Если полезны, то закодируйте их, добавьте к матрице с отскалированными признаками, обучите логистическую регрессию, замерьте качество.

In [ ]:
# -- YOUR CODE HERE --

**Ответ:** # -- YOUR ANSWER HERE --

Изучите признак TweetAt в обучающей выборке: преобразуйте его к типу datetime и нарисуйте его гистограмму с разделением по цвету на основе целевой переменной. Полезен ли он? Если полезен, то закодируйте его, добавьте к матрице с отскалированными признаками, обучите логистическую регрессию, замерьте качество.

In [ ]:
# -- YOUR CODE HERE --

**Ответ:** # -- YOUR ANSWER HERE --



Поработайте с признаком Location в обучающей выборке. Сколько уникальных значений?

In [ ]:
# -- YOUR CODE HERE --

Постройте гистограмму топ-10 по популярности местоположений (исключая Unknown)

In [ ]:
# -- YOUR CODE HERE --

Видно, что многие местоположения включают в себя более точное название места, чем другие (Например, у некоторых стоит London, UK; а у некоторых просто UK или United Kingdom).

Создайте новый признак WiderLocation, который содержит самое широкое местоположение (например, из London, UK должно получиться UK). Сколько уникальных категорий теперь? Постройте аналогичную гистограмму.

In [ ]:
# -- YOUR CODE HERE --

Закодируйте признак WiderLocation с помощью OHE таким образом, чтобы создались только столбцы для местоположений, которые встречаются более одного раза. Сколько таких значений?


In [ ]:
# -- YOUR CODE HERE --

Добавьте этот признак к матрице отскалированных текстовых признаков, обучите логистическую регрессию, замерьте качество. Как оно изменилось? Оказался ли признак полезным?


*Подсказка:* используйте параметр `categories` в энкодере.

In [ ]:
# -- YOUR CODE HERE --

**Ответ:** # -- YOUR ANSWER HERE --